In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

# Reload the data files from specified sheets
parameters_df = pd.read_excel('parameters_finaleee.xlsx', sheet_name='QP1200_2')
target_df = pd.read_excel('parameters_final.xlsx', sheet_name='theory_QP1200_2')

# Extract feature columns and target columns from the training data
features = parameters_df[['tau0', 'a', 'h0', 'tcs']]
targets = parameters_df[['peak', 'drop point', '25% peak', '25% drop', '50% peak', '50% drop', '75% peak', '75% drop']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Scale the features for optimal performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the multi-output regressor with Gradient Boosting as the base model
model = MultiOutputRegressor(GradientBoostingRegressor())

# Train the model on the scaled training data
model.fit(X_train_scaled, y_train)

# Predict on the test set and calculate the Mean Absolute Error (MAE) for performance
y_pred = model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, y_pred)

# Define the target outcomes based on the "theory" sheet data
target_outcomes = target_df.mean()  # Adjust as needed to align with actual theoretical target

tolerance = 0.05  # 5% tolerance

# Function to calculate if the predictions are within the tolerance range
def is_within_tolerance(predictions, targets, tolerance):
    # Check if all predictions are within (1 ± tolerance) range of the target outcomes
    lower_bound = targets * (1 - tolerance)
    upper_bound = targets * (1 + tolerance)
    return ((predictions >= lower_bound) & (predictions <= upper_bound)).all()

# Start with an initial set of parameters (mean of training data as a reasonable starting point)
current_params = X_train.mean().values.reshape(1, -1)
scaled_current_params = scaler.transform(current_params)  # Scale initial parameters

# Iterative optimization loop
max_iterations = 1000
for iteration in range(max_iterations):
    # Predict the outcomes for the current parameters
    predictions = model.predict(scaled_current_params).flatten()
    
    # Check if predictions are within tolerance
    if is_within_tolerance(predictions, target_outcomes.values, tolerance):
        optimal_params = scaler.inverse_transform(scaled_current_params)[0]  # Rescale to original parameters
        break
    
    # Adjust each parameter slightly and check if it improves prediction toward target
    improved = False
    for param_index in range(current_params.shape[1]):
        # Create a slight positive and negative adjustment to the current parameter
        for adjustment in [-0.01, 0.01]:  # Small adjustments
            test_params = scaled_current_params.copy()
            test_params[0, param_index] += adjustment
            test_predictions = model.predict(test_params).flatten()
            
            # Check if this adjustment brings predictions closer to the target
            if is_within_tolerance(test_predictions, target_outcomes.values, tolerance):
                optimal_params = scaler.inverse_transform(test_params)[0]
                improved = True
                break
        if improved:
            break
    else:
        # No improvement found with current adjustments; stop iterating
        optimal_params = scaler.inverse_transform(scaled_current_params)[0]
        break

# Display the final optimal parameters found
optimal_params

/opt/software/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([3.66071429e+02, 8.25000000e-01, 1.06250000e+04, 7.23214286e+02])

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --------------------------
# Data Loading and Preparation
# --------------------------
parameters_df = pd.read_excel('parameters_finaleee.xlsx', sheet_name='QP1200_2')
target_df = pd.read_excel('parameters_final.xlsx', sheet_name='theory_QP1200_2')

# Extract feature columns and target columns
features = parameters_df[['tau0', 'a', 'h0', 'tcs']]
targets = parameters_df[['peak', 'drop point', '25% peak', '25% drop', '50% peak', '50% drop', '75% peak', '75% drop']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Scale the features for optimal ANN performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --------------------------
# Build and Train the ANN Model
# --------------------------
# Define the neural network architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_train.shape[1])  # Output layer with one neuron per target
])

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Using MSE as a loss function

# Train the model
model.fit(X_train_scaled, y_train.values, epochs=100, batch_size=16, verbose=1)

# Evaluate model performance on the test set
y_pred = model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, y_pred)
print(f"Test MAE: {test_mae:.3f}")

# --------------------------
# Parameter Targeting Logic
# --------------------------
# Define the target outcomes based on the "theory" sheet data
# You may need to adjust how you define the target_outcomes.
target_outcomes = target_df.mean()  # Using the mean of theory data as the target

tolerance = 0.05  # 5% tolerance

def is_within_tolerance(predictions, targets, tolerance):
    # Check if all predictions are within (1 ± tolerance) of the target outcomes
    lower_bound = targets * (1 - tolerance)
    upper_bound = targets * (1 + tolerance)
    return ((predictions >= lower_bound) & (predictions <= upper_bound)).all()

# Start with an initial set of parameters (mean of training data as starting point)
current_params = X_train.mean().values.reshape(1, -1)
scaled_current_params = scaler.transform(current_params)  # Scale initial parameters

# Iterative optimization loop
max_iterations = 1000
optimal_params = None

for iteration in range(max_iterations):
    # Predict outcomes for current parameters
    predictions = model.predict(scaled_current_params).flatten()
    
    # Check tolerance
    if is_within_tolerance(predictions, target_outcomes.values, tolerance):
        optimal_params = scaler.inverse_transform(scaled_current_params)[0]
        print(f"Found parameters within tolerance at iteration {iteration}: {optimal_params}")
        break
    
    # Attempt to improve parameters
    improved = False
    for param_index in range(current_params.shape[1]):
        # Create slight positive and negative adjustments
        for adjustment in [-0.01, 0.01]:
            test_params = scaled_current_params.copy()
            test_params[0, param_index] += adjustment
            
            test_predictions = model.predict(test_params).flatten()
            if is_within_tolerance(test_predictions, target_outcomes.values, tolerance):
                optimal_params = scaler.inverse_transform(test_params)[0]
                print(f"Found parameters within tolerance after adjustment at iteration {iteration}: {optimal_params}")
                improved = True
                break
        if improved:
            break
    else:
        # If no improvement found, break out
        optimal_params = scaler.inverse_transform(scaled_current_params)[0]
        print(f"No improvement found after iteration {iteration}.")
        break

if optimal_params is not None:
    print("Optimal Parameters Found:", optimal_params)
else:
    print("No parameters found within tolerance.")


2024-12-12 15:32:08.106509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
2/2 [==============================] - 0s 7ms/step - loss: 0.0411
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0290
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0189
Epoch 4/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0126
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 6/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0049
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 0.0030
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 10/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 11/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 12/100
2/2 [==============================] - 0s 3ms/step - loss: 0.0016
Epoch 13/100
2/2 [==============================] - 0s 3ms/st

/opt/software/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 16ms/step
No improvement found after iteration 0.
Optimal Parameters Found: [3.66071429e+02 8.25000000e-01 1.06250000e+04 7.23214286e+02]
